In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the dataset
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# Extract columns that represent monthly units
monthly_columns = [col for col in data.columns if '_unit' in col]

# Calculate the average monthly consumption across all customers
average_monthly_consumption = data[monthly_columns].apply(pd.to_numeric, errors='coerce').mean()

# Extract months for labeling in chronological order
months_mod = [
    col.replace('_unit', '')
       .replace('pre_', 'Prepaid: ')
       .replace('post_', 'Postpaid: ')
       .replace('21', '2021')
       .replace('22', '2022')
       .replace('23', '2023')
       .replace('_', ' ')
       .title()
    for col in monthly_columns
]

# Swap the position of month and year to be "Month Year"
months = [
    ' '.join([part.strip() for part in label.split(':')[1].split()[::-1]]) + f" ({label.split(':')[0]})"
    for label in months_mod
]

years = ['2021', '2022', '2023']

# Plot the time series of average monthly consumption
plt.figure(figsize=(10, 6))
plt.plot(months, average_monthly_consumption, marker='o', linestyle='-', linewidth=2)
plt.xticks(rotation=45, ha='right')
plt.title('Average Monthly Consumption Over Time')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.grid(True)
plt.tight_layout()
plt.savefig('average_monthly_consumption.png', dpi=300)
plt.show()

# Corrected Box Plot: Monthly consumption distribution
plt.figure(figsize=(12, 6))
sns.boxplot(data=data[monthly_columns].apply(pd.to_numeric, errors='coerce'), showfliers=False)
plt.xticks(ticks=np.arange(len(months)), labels=months, rotation=45, ha='right')
plt.title('Monthly Consumption Distribution (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Consumption (Units)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig('monthly_consumption_distribution.png', dpi=300)
plt.show()

# Yearly Trend Comparison
formatted_months = months
yearly_data = pd.DataFrame(index=formatted_months)
yearly_data['Average Consumption'] = average_monthly_consumption.values

# Line plot for yearly comparison
plt.figure(figsize=(10, 6))
for year in years:
    year_data = yearly_data[yearly_data.index.str.contains(year)]
    plt.plot(year_data.index, year_data['Average Consumption'], marker='o', label=f'{year}')

plt.xticks(rotation=45, ha='right')
plt.title('Yearly Average Monthly Consumption Comparison (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.legend(title='Year')
plt.grid(True)
plt.tight_layout()
plt.savefig('yearly_average_comparison.png', dpi=300)
plt.show()

# Suggested New Visualizations
# 1. Consumption Trend with Moving Average
plt.figure(figsize=(10, 6))
rolling_avg = average_monthly_consumption.rolling(window=3, min_periods=1).mean()
plt.plot(months, average_monthly_consumption, marker='o', linestyle='-', label='Monthly Average')
plt.plot(months, rolling_avg, marker='o', linestyle='--', color='orange', label='3-Month Moving Average')
plt.xticks(rotation=45, ha='right')
plt.title('Average Monthly Consumption with 3-Month Moving Average')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('average_monthly_with_moving_average.png', dpi=300)
plt.show()

# 2. Monthly Consumption Distribution by Year (Violin Plot)
plt.figure(figsize=(12, 6))
data_melted = pd.melt(data[monthly_columns].apply(pd.to_numeric, errors='coerce'), var_name='Month', value_name='Consumption')
data_melted['Year'] = data_melted['Month'].str.extract('(21|22|23)').replace({'21': '2021', '22': '2022', '23': '2023'})
data_melted['Month'] = data_melted['Month'].str.replace('pre_|post_|_unit', '').str.title()
sns.violinplot(x='Month', y='Consumption', hue='Year', data=data_melted, split=True)
plt.xticks(rotation=45, ha='right')
plt.title('Monthly Consumption Distribution by Year (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Consumption (Units)')
plt.tight_layout()
plt.savefig('monthly_consumption_violinplot.png', dpi=300)
plt.show()

# 3. Scatter Plot for Prepaid vs Postpaid Consumption (Monthly)
plt.figure(figsize=(10, 6))
prepaid_values = data[monthly_columns].filter(like='pre_').apply(pd.to_numeric, errors='coerce').mean().values
postpaid_values = data[monthly_columns].filter(like='post_').apply(pd.to_numeric, errors='coerce').mean().values

# Ensure both arrays are the same length by trimming to the minimum length
min_length = min(len(prepaid_values), len(postpaid_values))
prepaid_values = prepaid_values[:min_length]
postpaid_values = postpaid_values[:min_length]

plt.scatter(prepaid_values, postpaid_values, c='green', edgecolors='black', alpha=0.6)
plt.plot([min(prepaid_values), max(prepaid_values)], [min(postpaid_values), max(postpaid_values)], 'r--')
plt.xlabel('Prepaid Consumption (Units)')
plt.ylabel('Postpaid Consumption (Units)')
plt.title('Prepaid vs Postpaid Consumption (Monthly)')
plt.grid(True)
plt.tight_layout()
plt.savefig('prepaid_vs_postpaid_scatter.png', dpi=300)
plt.show()

# 4. Heatmap of Average Monthly Consumption by Year and Type
avg_data = pd.DataFrame({'Prepaid': prepaid_values, 'Postpaid': postpaid_values}, index=months[:min_length])
plt.figure(figsize=(10, 6))
sns.heatmap(avg_data.T, annot=True, cmap='Blues', linewidths=0.5, fmt='.2f')
plt.title('Average Monthly Consumption by Year and Type')
plt.xlabel('Month')
plt.ylabel('Type')
plt.tight_layout()
plt.savefig('average_consumption_heatmap.png', dpi=300)
plt.show()

# 5. Scatter Plot for AC vs Units
# plt.figure(figsize=(10, 6))
# if 'AC' in data.columns:
#     ac_consumption = pd.to_numeric(data['AC'], errors='coerce')
#     total_units = data[monthly_columns].apply(pd.to_numeric, errors='coerce').sum(axis=1)
#     plt.scatter(ac_consumption, total_units, c='blue', edgecolors='black', alpha=0.6)
#     plt.xlabel('AC Consumption (Units)')
#     plt.ylabel('Total Consumption (Units)')
#     plt.title('AC Consumption vs Total Consumption (Units)')
#     plt.grid(True)
#     plt.tight_layout()
#     plt.savefig('ac_vs_total_consumption.png', dpi=300)
#     plt.show()
# else:
#     print("Column 'AC' not found in the dataset.")

# 6. Bar Plot of Household Appliances vs Total Consumption
appliance_columns = ['Rooms', 'Members', 'Fans', 'Lights', 'Freezes', 'Microwave Oven', 'Geyser', 'AC', 'Washing Machine', 'TV', 'PC', 'Induction Stove']

# Check if all appliance columns exist in the dataset
existing_appliance_columns = [col for col in appliance_columns if col in data.columns]

# Calculate the correlation of each appliance with total consumption
total_units = data[monthly_columns].apply(pd.to_numeric, errors='coerce').sum(axis=1)
correlations = data[existing_appliance_columns].apply(pd.to_numeric, errors='coerce').corrwith(total_units)

plt.figure(figsize=(12, 6))
sns.barplot(x=correlations.index, y=correlations.values, hue=correlations.index, palette='viridis', dodge=False, legend=False)
plt.xticks(rotation=45, ha='right')
plt.title('Correlation of Household Appliances with Total Consumption')
plt.xlabel('Appliance')
plt.ylabel('Correlation with Total Consumption')
plt.tight_layout()
plt.savefig('appliance_vs_total_consumption.png', dpi=300)
plt.show()

In [ ]:

# 5. Analysis of Appliance Consumption Correlation with Total Consumption
# Group customers by the number of AC, Geyser, Induction Stove, and Rooms units and compare total consumption
appliances = ['Rooms', 'Members', 'Fans', 'Lights', 'Freezes', 'Microwave Oven',
                           'Geyser', 'AC', 'Washing Machine', 'TV', 'PC', 'Induction Stove']
total_units = data[monthly_columns].apply(pd.to_numeric, errors='coerce').sum(axis=1)
data['Total_Units'] = total_units

for appliance in appliances:
    plt.figure(figsize=(12, 8))
    appliance_consumption = pd.to_numeric(data[appliance], errors='coerce').fillna(0)
    data[appliance] = appliance_consumption
    appliance_groups = data.groupby(appliance)['Total_Units'].mean().sort_index()

    # Plot the average consumption for each appliance count group
    sns.barplot(x=appliance_groups.index, y=appliance_groups.values, hue=appliance_groups.index, palette='viridis', dodge=False, legend=False)
    plt.xlabel(f'Number of {appliance} Units')
    plt.ylabel('Average Total Consumption (Units)')
    plt.title(f'Average Total Consumption by Number of {appliance} Units')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{appliance.lower()}_vs_total_consumption_correlation.png', dpi=300)
    plt.show()

    # Show consumption distribution for different appliance groups
    plt.figure(figsize=(12, 8))
    for count in sorted(appliance_groups.index):
        subset = data[data[appliance] == count]
        sns.kdeplot(subset['Total_Units'], label=f'{int(count)} {appliance}(s)', fill=True)

    plt.xlabel('Total Consumption (Units)')
    plt.ylabel('Density')
    plt.title(f'Consumption Distribution by Number of {appliance} Units')
    plt.legend(title=f'Number of {appliance} Units')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{appliance.lower()}_consumption_distribution.png', dpi=300)
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the dataset
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# Extract columns that represent monthly units
monthly_columns = [col for col in data.columns if '_unit' in col]

# Calculate the average monthly consumption across all customers
average_monthly_consumption = data[monthly_columns].apply(pd.to_numeric, errors='coerce').mean()

# Extract months for labeling in chronological order
months_mod = [
    col.replace('_unit', '')
       .replace('pre_', 'Prepaid: ')
       .replace('post_', 'Postpaid: ')
       .replace('21', '2021')
       .replace('22', '2022')
       .replace('23', '2023')
       .replace('_', ' ')
       .title()
    for col in monthly_columns
]

# Swap the position of month and year to be "Month Year"
months = [
    ' '.join([part.strip() for part in label.split(':')[1].split()[::-1]]) + f" ({label.split(':')[0]})"
    for label in months_mod
]

years = ['2021', '2022', '2023']

# Plot the time series of average monthly consumption
plt.figure(figsize=(10, 6))
plt.plot(months, average_monthly_consumption, marker='o', linestyle='-', linewidth=2)
plt.xticks(rotation=45, ha='right')
plt.title('Average Monthly Consumption Over Time')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.grid(True)
plt.tight_layout()
plt.savefig('average_monthly_consumption.png', dpi=300)
plt.show()

# Corrected Box Plot: Monthly consumption distribution
plt.figure(figsize=(12, 6))
sns.boxplot(data=data[monthly_columns].apply(pd.to_numeric, errors='coerce'), showfliers=False)
plt.xticks(ticks=np.arange(len(months)), labels=months, rotation=45, ha='right')
plt.title('Monthly Consumption Distribution (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Consumption (Units)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig('monthly_consumption_distribution.png', dpi=300)
plt.show()

# Yearly Trend Comparison
formatted_months = months
yearly_data = pd.DataFrame(index=formatted_months)
yearly_data['Average Consumption'] = average_monthly_consumption.values

# Line plot for yearly comparison
plt.figure(figsize=(10, 6))
for year in years:
    year_data = yearly_data[yearly_data.index.str.contains(year)]
    plt.plot(year_data.index, year_data['Average Consumption'], marker='o', label=f'{year}')

plt.xticks(rotation=45, ha='right')
plt.title('Yearly Average Monthly Consumption Comparison (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.legend(title='Year')
plt.grid(True)
plt.tight_layout()
plt.savefig('yearly_average_comparison.png', dpi=300)
plt.show()

# Suggested New Visualizations
# 1. Consumption Trend with Moving Average
plt.figure(figsize=(10, 6))
rolling_avg = average_monthly_consumption.rolling(window=3, min_periods=1).mean()
plt.plot(months, average_monthly_consumption, marker='o', linestyle='-', label='Monthly Average')
plt.plot(months, rolling_avg, marker='o', linestyle='--', color='orange', label='3-Month Moving Average')
plt.xticks(rotation=45, ha='right')
plt.title('Average Monthly Consumption with 3-Month Moving Average')
plt.xlabel('Month')
plt.ylabel('Average Consumption (Units)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('average_monthly_with_moving_average.png', dpi=300)
plt.show()

# 2. Monthly Consumption Distribution by Year (Violin Plot)
plt.figure(figsize=(12, 6))
data_melted = pd.melt(data[monthly_columns].apply(pd.to_numeric, errors='coerce'), var_name='Month', value_name='Consumption')
data_melted['Year'] = data_melted['Month'].str.extract('(21|22|23)').replace({'21': '2021', '22': '2022', '23': '2023'})
data_melted['Month'] = data_melted['Month'].str.replace('pre_|post_|_unit', '').str.title()
sns.violinplot(x='Month', y='Consumption', hue='Year', data=data_melted, split=True)
plt.xticks(rotation=45, ha='right')
plt.title('Monthly Consumption Distribution by Year (Prepaid and Postpaid)')
plt.xlabel('Month')
plt.ylabel('Consumption (Units)')
plt.tight_layout()
plt.savefig('monthly_consumption_violinplot.png', dpi=300)
plt.show()

# 3. Scatter Plot for Prepaid vs Postpaid Consumption (Monthly)
plt.figure(figsize=(10, 6))
prepaid_values = data[monthly_columns].filter(like='pre_').apply(pd.to_numeric, errors='coerce').mean().values
postpaid_values = data[monthly_columns].filter(like='post_').apply(pd.to_numeric, errors='coerce').mean().values

# Ensure both arrays are the same length by trimming to the minimum length
min_length = min(len(prepaid_values), len(postpaid_values))
prepaid_values = prepaid_values[:min_length]
postpaid_values = postpaid_values[:min_length]

plt.scatter(prepaid_values, postpaid_values, c='green', edgecolors='black', alpha=0.6)
plt.plot([min(prepaid_values), max(prepaid_values)], [min(postpaid_values), max(postpaid_values)], 'r--')
plt.xlabel('Prepaid Consumption (Units)')
plt.ylabel('Postpaid Consumption (Units)')
plt.title('Prepaid vs Postpaid Consumption (Monthly)')
plt.grid(True)
plt.tight_layout()
plt.savefig('prepaid_vs_postpaid_scatter.png', dpi=300)
plt.show()

# 4. Heatmap of Average Monthly Consumption by Year and Type
avg_data = pd.DataFrame({'Prepaid': prepaid_values, 'Postpaid': postpaid_values}, index=months[:min_length])
plt.figure(figsize=(10, 6))
sns.heatmap(avg_data.T, annot=True, cmap='Blues', linewidths=0.5, fmt='.2f')
plt.title('Average Monthly Consumption by Year and Type')
plt.xlabel('Month')
plt.ylabel('Type')
plt.tight_layout()
plt.savefig('average_consumption_heatmap.png', dpi=300)
plt.show()

# 5. Bar Plot of Household Appliances vs Total Consumption
appliance_columns = ['Rooms', 'Members', 'Fans', 'Lights', 'Freezes', 'Microwave Oven', 'Geyser', 'AC', 'Washing Machine', 'TV', 'PC', 'Induction Stove']

# Check if all appliance columns exist in the dataset
existing_appliance_columns = [col for col in appliance_columns if col in data.columns]

# Calculate the correlation of each appliance with total consumption
total_units = data[monthly_columns].apply(pd.to_numeric, errors='coerce').sum(axis=1)
correlations = data[existing_appliance_columns].apply(pd.to_numeric, errors='coerce').corrwith(total_units)

plt.figure(figsize=(12, 6))
sns.barplot(x=correlations.index, y=correlations.values, hue=correlations.index, palette='viridis', dodge=False, legend=False)
plt.xticks(rotation=45, ha='right')
plt.title('Correlation of Household Appliances with Total Consumption')
plt.xlabel('Appliance')
plt.ylabel('Correlation with Total Consumption')
plt.tight_layout()
plt.savefig('appliance_vs_total_consumption.png', dpi=300)
plt.show()

# Analysis of Appliance Consumption Correlation with Total Consumption
# Group customers by the number of AC, Geyser, Induction Stove, and Rooms units and compare total consumption
appliances = ['Rooms', 'Members', 'Fans', 'Lights', 'Freezes', 'Microwave Oven',
                           'Geyser', 'AC', 'Washing Machine', 'TV', 'PC', 'Induction Stove']
total_units = data[monthly_columns].apply(pd.to_numeric, errors='coerce').sum(axis=1)
data['Total_Units'] = total_units

for appliance in appliances:
    plt.figure(figsize=(12, 8))
    appliance_consumption = pd.to_numeric(data[appliance], errors='coerce').fillna(0)
    data[appliance] = appliance_consumption
    appliance_groups = data.groupby(appliance)['Total_Units'].mean().sort_index()

    # Plot the average consumption for each appliance count group
    sns.barplot(x=appliance_groups.index, y=appliance_groups.values, hue=appliance_groups.index, palette='viridis', dodge=False, legend=False)
    plt.xlabel(f'Number of {appliance} Units')
    plt.ylabel('Average Total Consumption (Units)')
    plt.title(f'Average Total Consumption by Number of {appliance} Units')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{appliance.lower()}_vs_total_consumption_correlation.png', dpi=300)
    plt.show()

    # Show consumption distribution for different appliance groups
    plt.figure(figsize=(12, 8))
    for count in sorted(appliance_groups.index):
        subset = data[data[appliance] == count]
        if len(subset) > 1:  # Avoid KDE for single or no data points
            sns.kdeplot(subset['Total_Units'], label=f'{int(count)} {appliance}(s)', fill=True, warn_singular=False)

    plt.xlabel('Total Consumption (Units)')
    plt.ylabel('Density')
    plt.title(f'Consumption Distribution by Number of {appliance} Units')
    plt.legend(title=f'Number of {appliance} Units')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{appliance.lower()}_consumption_distribution.png', dpi=300)
    plt.show()

In [ ]:
# Room, member, fan , light, freeze 


zero_rooms_data = data[data['Rooms'] == 0]
# Print the count of rows with zero rooms and their 'Customer No' and 'Total_Units'
zero_rooms_summary = zero_rooms_data[['Customer No', 'Rooms', 'Total_Units', "Serial" ]]
zero_rooms_summary.head(), len(zero_rooms_summary)


In [ ]:
zero_rooms_data = data[data['Members'] == 0]
# Print the count of rows with zero rooms and their 'Customer No' and 'Total_Units'
zero_rooms_summary = zero_rooms_data[['Customer No', 'Members', 'Total_Units', "Serial" ]]
zero_rooms_summary.head(), len(zero_rooms_summary)


In [ ]:
zero_rooms_data = data[data['Fans'] == 0]
# Print the count of rows with zero rooms and their 'Customer No' and 'Total_Units'
zero_rooms_summary = zero_rooms_data[['Customer No', 'Fans', 'Total_Units', "Serial" ]]
zero_rooms_summary.head(), len(zero_rooms_summary)

In [ ]:

zero_rooms_data = data[data['Lights'] == 0]
# Print the count of rows with zero rooms and their 'Customer No' and 'Total_Units'
zero_rooms_summary = zero_rooms_data[['Customer No', 'Lights', 'Total_Units', "Serial" ]]
zero_rooms_summary.head(), len(zero_rooms_summary)

In [ ]:

zero_rooms_data = data[data['Freezes'] == 0]
# Print the count of rows with zero rooms and their 'Customer No' and 'Total_Units'
zero_rooms_summary = zero_rooms_data[['Customer No', 'Freezes', 'Total_Units', "Serial" ]]
zero_rooms_summary.head(), len(zero_rooms_summary)